In [1]:
import xml.etree.ElementTree as ET

In [2]:
import wave
import numpy as np
import struct
import sys
import csv
import resampy
from scipy.io import wavfile
from scipy.signal import resample
import pandas as pd
import numpy
import random
import sys
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
%matplotlib inline

In [3]:
def write_wav(data, filename, framerate, amplitude):
    wavfile = wave.open(filename,'w')
    nchannels = 1
    sampwidth = 2
    framerate = framerate
    nframes = len(data)
    comptype = "NONE"
    compname = "not compressed"
    wavfile.setparams((nchannels,
                        sampwidth,
                        framerate,
                        nframes,
                        comptype,
                        compname))
    frames = []
    for s in data:
        mul = int(s * amplitude)
        frames.append(struct.pack('h', mul))

    frames = ''.join(frames)
    wavfile.writeframes(frames)
    wavfile.close()
    print("%s written" %(filename)) 

In [4]:
def xml2csv(texture, attrib):
    tree = ET.parse(attrib + "_" + texture + ".xml")
    root = tree.getroot()
    accel = root.find(attrib)
    csvlist = [[attrib]]
    for value in accel:
        tmp = [value.text]
        csvlist.append(tmp)
    csvname = attrib + "_" + texture + ".csv"
    with open(csvname, "w", encoding="Shift_jis") as f:
        writer = csv.writer(f, lineterminator="\n")
        writer.writerows(csvlist)
    return csvname

In [5]:
def csv2wav(csvname, wavname):
    df = pd.read_csv(csvname)
    data = df.values
    data /= numpy.max(numpy.abs(data))
    data_resampled = resample(data, len(data))
    wavfile.write(wavname, 10000, data_resampled)

In [6]:
#import pysoundfile as sf
def list2wav(data, wavname):
    data = numpy.array(data, dtype='float64')
    data /= numpy.max(numpy.abs(data))
    data *= 2147483647
    #data_resampled = resample(data, len(data))
    data = data.astype(np.int32)
    wavfile.write(wavname, 10000, data)

In [7]:
def createcsv(data, csvname):
    with open(csvname, "w", encoding="Shift_jis") as f:
        writer = csv.writer(f, lineterminator="\n")
        writer.writerows(data)

In [8]:
def getParams(xmlname):
    tree = ET.parse(xmlname)
    root = tree.getroot()
    numMod = int(root.find('numMod').text)
    numARCoeff = int(root.find('numARCoeff').text)
    numMACoeff = int(root.find('numMACoeff').text)
    yield numMod, numARCoeff, numMACoeff
    for model in root:
        if(model.tag != 'model'):
            continue
        modelnum = model.find('modNum').text
        speed = model.find('speedMod').text
        ARcoeff = []
        for value in model.find('ARcoeff'):
            ARcoeff.append(value.text)
        #tmp.append(model.find('var').text)
        ARcoeff.append('1.0')
        MAcoeff = []
        var = float(model.find('var').text)
        gain = float(model.find('gain').text)
        for value in model.find('MAcoeff'):
            MAcoeff.append(value.text)
        #tmp.append(model.find('var').text)
        #MAcoeff.append('1.0')
        yield modelnum, speed, ARcoeff, MAcoeff, var, gain

In [9]:
import math
def createARMAmodel(histIn, histOut, ARcoeff, MAcoeff,ARorder, MAorder, var):
    ARcoeff = numpy.array(ARcoeff, dtype='float64').T   #Coefficent of AR model
    MAcoeff = numpy.array(MAcoeff, dtype='float64').T   #Coefficein of MA model
    histOut[ARorder + 1] = random.random()#.normal(0, math.sqrt(var))            #Create Input of time t
    y_t = numpy.dot(histOut, ARcoeff) + numpy.dot(histIn, MAcoeff) #Create Output of time t
    #preserve params of this step
    if(MAorder != 0):
        histIn[2:MAorder] = histIn[1:MAorder-1]
        histIn[1] = histOut[ARorder+1]
    histOut[2:ARorder+1] = histOut[1:ARorder]
    histOut[1] = y_t
    return y_t, histOut, histIn

In [10]:
import math
def createARMAmodel2(histIn, histOut, ARcoeff, MAcoeff,ARorder, MAorder, var, gain):
    ARcoeff = numpy.array(ARcoeff, dtype='float64').T   #Coefficent of AR model
    MAcoeff = numpy.array(MAcoeff, dtype='float64').T   #Coefficein of MA model
    if(var == 0.0):
        histOut[ARorder] = numpy.random.normal(0, 1)            #Create Input of time
    else:
        histOut[ARorder] = numpy.random.normal(0, math.sqrt(var)) * gain            #Create Input of time t
    y_t = -numpy.dot(histOut, ARcoeff) + numpy.dot(histIn, MAcoeff) * gain #Create Output of time t
    #preserve params of this step
    if(MAorder != 0):
        histIn[2:MAorder] = histIn[1:MAorder-1]
        histIn[1] = histOut[ARorder]
    histOut[1:ARorder] = histOut[:ARorder-1]
    histOut[1] = y_t
    return y_t, histOut, histIn

In [11]:
from scipy.signal import buttord,butter,filtfilt
def butter_filter(data, samplingrate, cutoff, order, fitertype='low'):
    fnyq = 0.5 * samplingrate
    normal_cutoff = cutoff / fnyq
   # ws = fs / fnyq
    # Get the filter coefficients
    #N, Wn = buttord(wp, ws, gpass, gstop)
    b, a = butter(order, normal_cutoff, btype=fitertype, analog=False)
    y = filtfilt(b, a, data, padlen=0)
    return y

In [12]:
import glob
def fileiter():
    files = glob.glob("./*.xml")
    #yield len(files)
    for file in files:
        yield file

In [13]:
samplingrate = 100000 
T = 50000   #Number of points
high_cutoff = 1000
low_cutoff = 10
Polyorder = 1 #
files = fileiter()
Numfile = 100
for i in tqdm(range(Numfile)):
    xmlname = files.__next__()
    dirname = xmlname.split(".")[1][1:]
    model = getParams(xmlname)
    numMod, numARcoeff, numMAcoeff = model.__next__()
    for i in range(numMod):
        modelnum, speed, ARcoeff, MAcoeff, var, gain = model.__next__()
        ARcoeff = ARcoeff[1:]
        histIn = numpy.zeros(numMAcoeff + 1, dtype='float64')
        histOut = numpy.zeros(numARcoeff + 1, dtype='float64')
        #histOut = numpy.append(numpy.ones(1, dtype='float64'), numpy.zeros(numARcoeff + 1, dtype='float64'))
        data =[]
        #data 作る
        for i in range(T):
            (y_t, histOut, histIn) = createARMAmodel2(histIn, histOut, ARcoeff, MAcoeff, numARcoeff, numMAcoeff, var, gain)
            data.append(y_t)
        #csvファイル作る（パス，データ）
        csvname = './' + dirname + '/csv/' + 'Model' + modelnum + '.csv'
        #createcsv(data, csvname)
        #wav作る
        #data = data[10000:40001]
        #filterdata = butter_filter(data, samplingrate, high_cutoff, Polyorder, fitertype='low')
        #data = butter_filter(data, samplingrate, low_cutoff, Polyorder, fitertype='high')
        if(data[0] == data[0]):
            if(not(os.path.exists(dirname) or os.path.isdir(dirname))):
                os.mkdir(dirname)
            if(not(os.path.exists(dirname + "/csv") or os.path.isdir(dirname + "/csv"))):
               os.mkdir(dirname + "/csv")
            if(not(os.path.exists(dirname + "/wav") or os.path.isdir(dirname + "/wav"))):
               os.mkdir(dirname + "/wav")
            wavname = './' + dirname + '/wav/Speed' + speed + ".wav"
            list2wav(data, wavname)
            

  3%|██▍                                                                             | 3/100 [02:11<1:13:17, 45.34s/it]C:\Users\ymtmy\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:16:36<00:00, 45.96s/it]


In [ ]:
samplingrate = 100000 
T = 50000   #Number of points
high_cutoff = 1000
low_cutoff = 50
Polyorder = 1 #
Numfile = 100
xmlname = "Models_Aluminum Foil.xml"
dirname = xmlname.split(".")[0]
model = getParams(xmlname)
numMod, numARcoeff, numMAcoeff = model.__next__()
for i in tqdm(range(1)):
    modelnum, speed, ARcoeff, MAcoeff, var = model.__next__()
    #ARcoeff.reverse()
    #tmp = MAcoeff
    #tmp.reverse()
   # MAcoeff[1:] = tmp[:numMAcoeff]
    #MAcoeff[0] = 1.0
   # print(MAcoeff)
    histIn = numpy.zeros((numMAcoeff + 1), dtype='float64')
    histOut = numpy.append(numpy.ones(1, dtype='float64'), numpy.zeros(numARcoeff + 1, dtype='float64'))
    data =[]
    #data 作る
    for i in range(T):
        (y_t, histOut, histIn) = createARMAmodel(histIn, histOut, ARcoeff, MAcoeff, numARcoeff, numMAcoeff, var)
        data.append(y_t)
    #csvファイル作る（パス，データ）
    csvname = './' + dirname + '/csv/' + 'Model' + modelnum + '.csv'
     #createcsv(data, csvname)
    #wav作る
    data = data[10000:40001]
    #lowdata = butter_filter(data, samplingrate, high_cutoff, Polyorder, fitertype='low')
    if(lowdata[0] == lowdata[0]):
        banddata = butter_filter(data, samplingrate, low_cutoff, Polyorder, fitertype='high')
        #print(data[0:3])
        if(not(os.path.exists(dirname) or os.path.isdir(dirname))):
            os.mkdir(dirname)
        if(not(os.path.exists(dirname + "/csv") or os.path.isdir(dirname + "/csv"))):
            os.mkdir(dirname + "/csv")
        if(not(os.path.exists(dirname + "/wav2") or os.path.isdir(dirname + "/wav"))):
            os.mkdir(dirname + "/wav")
        wavname = './' + dirname + '/wav/Speed' + speed + ".wav"
        list2wav(banddata, wavname)
    #else:
    #     wavname = './' + dirname + '/wav/Nofilter_Speed' + speed + ".wav"
    #     list2wav(data, wavname)
        

In [16]:
numpy.random.normal(0,0 ,10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])